In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from slack import SlackInteractor
from claude_llm import ClaudeLLM
from project_manager_agent import ProjectManagerAgent
from sarcastic_agent import SarcasticAgent
from db import ActionDatabase
from runner import process_threads, execute_due_actions

In [3]:
slack_interactor = SlackInteractor()
llm = ClaudeLLM()
action_db = ActionDatabase()

agents = [
    ProjectManagerAgent(llm, action_db, slack_interactor),
    SarcasticAgent(llm, action_db, slack_interactor)
]

In [4]:
print("\nFetching new messages...")
data = slack_interactor.fetch_new_messages()
threads = slack_interactor.organize_threads(data)
print(f"Found {len(threads)} threads with new messages.")

results = process_threads(agents, threads)

print("\nChecking for due actions...")
execute_due_actions(agents)


Fetching new messages...
Set conversations_oldest to 1722399936.894109
Loaded 360 old messages


NameError: name 'np' is not defined